## Рекомендательные системы

### Описание задачи
Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.  
Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

***Входные данные***

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.  
В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.  
Гарантируется, что среди id купленных товаров все различные.

***Важно:***
* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

In [1]:
import pandas as pd
import numpy as np
import collections
import warnings

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("data/coursera_sessions_train.txt", sep=';', header=None, names=["seen", "bought"])
train.head()

,seen,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [3]:
test = pd.read_csv("data/coursera_sessions_test.txt", sep=';', header=None, names=["seen", "bought"]).dropna()
test.head()

,seen,bought
7,"63,68,69,70,66,61,59,61,66,68","66,63"
14,"158,159,160,159,161,162",162
19,"200,201,202,203,204","201,205"
34,"371,372,371","371,373"
40,422,422


**1.** На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

In [4]:
seen_frequences = pd.Series(','.join(train["seen"].values).split(',')).value_counts()
train.dropna(inplace=True)
bought_frequences = pd.Series(','.join(train["bought"].values).split(',')).value_counts()

**2.** Реализуйте два алгоритма рекомендаций:

2.1 сортировка просмотренных id по популярности (частота появления в просмотренных)

In [5]:
train["rec_by_seen"] = train.apply(
    lambda x: list(pd.DataFrame(seen_frequences.reindex(pd.unique(x["seen"].split(','))).fillna(0)).sort_values(by=0, ascending=False, kind='mergesort').index),
    axis=1
    )
test["rec_by_seen"] = test.apply(
    lambda x: list(pd.DataFrame(seen_frequences.reindex(pd.unique(x["seen"].split(','))).fillna(0)).sort_values(by=0, ascending=False, kind='mergesort').index),
    axis=1
    )
train.head()

,seen,bought,rec_by_seen
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 60, 61, 65, 66, 67, 68, 59, 62]"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 84, 92, 86, 87, 91, 88]"
19,"138,198,199,127",199,"[127, 138, 198, 199]"
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 304, 307, 309, 310, 305, 308, 311, ..."
33,"352,353,352",352,"[352, 353]"


2.2 сортировка просмотренных id по покупаемости (частота появления в покупках).

In [6]:
train["rec_by_bought"] = train.apply(
    lambda x: list(pd.DataFrame(bought_frequences.reindex(pd.unique(x["seen"].split(','))).fillna(0)).sort_values(by=0, ascending=False, kind='mergesort').index),
    axis=1)
test["rec_by_bought"] = test.apply(
    lambda x: list(pd.DataFrame(bought_frequences.reindex(pd.unique(x["seen"].split(','))).fillna(0)).sort_values(by=0, ascending=False, kind='mergesort').index),
    axis=1)
train.head()

,seen,bought,rec_by_seen,rec_by_bought
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 60, 61, 65, 66, 67, 68, 59, 62]","[60, 63, 67, 59, 61, 62, 64, 65, 66, 68]"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 84, 92, 86, 87, 91, 88]","[86, 85, 93, 84, 87, 88, 89, 90, 91, 92]"
19,"138,198,199,127",199,"[127, 138, 198, 199]","[138, 199, 127, 198]"
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 304, 307, 309, 310, 305, 308, 311, ...","[303, 304, 305, 306, 307, 308, 309, 310, 311, ..."
33,"352,353,352",352,"[352, 353]","[352, 353]"


**3.** Для данных алгоритмов выпишите через пробел **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом.   
Где качество получилось выше?  
Значимо ли это различие?  
Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.  

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)


In [7]:
def get_average_recall_precision_k(dataset, recomend_by_column, k_array):
    average_recall_precision = []
    for k in k_array:
        average_recall_precision.append(np.round(dataset.apply(lambda row:
                                                                    len(set.intersection(set(row["bought"].split(',')),
                                                                                            set(row[recomend_by_column][:k])))
                                                                    /(len(row["bought"].split(','))),
                                                                axis=1).mean(), 2))
        average_recall_precision.append(np.round(dataset.apply(lambda row: 
                                                                    len(set.intersection(set(row["bought"].split(',')),
                                                                                         set(row[recomend_by_column][:k]))) /
                                                                    k,
                                                                axis=1).mean(), 2))
    return average_recall_precision

In [8]:
def save_answerArray(fname,array):
    with open(fname,"w") as fout:
        fout.write(" ".join([str(el) for el in array]))

In [9]:
datasets = {"train" : train,
            "test" : test}
rec_by_array = ["rec_by_seen", "rec_by_bought"]
k_array = [1, 5]

for dataset in datasets:
    for rec_by_column in rec_by_array:
        average_recall_precision_k = get_average_recall_precision_k(datasets[dataset], rec_by_column, k_array)
        save_answerArray(f"answers\w4_task_1\\{dataset}_{rec_by_column}.txt", average_recall_precision_k)    
        print(f"Report on {dataset}, {rec_by_column}:\n \
        AverageRecall@1 = {average_recall_precision_k[0]}\n \
        AveragePrecision@1 = {average_recall_precision_k[1]}\n \
        AverageRecall@5 = {average_recall_precision_k[2]}\n \
        AveragePrecision@5 = {average_recall_precision_k[3]}")


Report on train, rec_by_seen:
         AverageRecall@1 = 0.44
         AveragePrecision@1 = 0.51
         AverageRecall@5 = 0.82
         AveragePrecision@5 = 0.21
Report on train, rec_by_bought:
         AverageRecall@1 = 0.69
         AveragePrecision@1 = 0.8
         AverageRecall@5 = 0.93
         AveragePrecision@5 = 0.25
Report on test, rec_by_seen:
         AverageRecall@1 = 0.42
         AveragePrecision@1 = 0.48
         AverageRecall@5 = 0.8
         AveragePrecision@5 = 0.2
Report on test, rec_by_bought:
         AverageRecall@1 = 0.46
         AveragePrecision@1 = 0.53
         AverageRecall@5 = 0.82
         AveragePrecision@5 = 0.21


**Вывод**: Рекомендательная система, которая учитывает частоту покупок товаров для просмотренных даёт результат выше, чем система, которая рекомендует просмотренные товары по их популярности. И здесь можно найти логическое обоснование, люди посмотрев некоторое число товаров, вероятней всего придут к покупки того же товара, что и другие люди, которые посмотрели такие же (или подобные) товары.